In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import gc

import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import math
import lightgbm as lgb
from tqdm.notebook import tqdm
tqdm.pandas()

#!pip install Wordbatch
#from wordbatch.models import FTRL, FM_FTRL  # https://github.com/anttttti/Wordbatch
import xgboost
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.model_selection import train_test_split
import scipy
from scipy.sparse import hstack

from sklearn.metrics import mean_squared_log_error
import pickle
import regex as re
import os

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/My Drive/Colab Notebooks/'

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# if 'train.tsv' not in os.listdir(): 
#     !kaggle competitions download -c mercari-price-suggestion-challenge
#     get_ipython().system_raw("7z x \*.7z && rm *.7z")
#     get_ipython().system_raw("7z x \*.zip && rm *.zip")

# Unzip data files
!apt install --assume-yes p7zip-full
!7z x ../input/mercari-price-suggestion-challenge/train.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip -y
!7z x ../input/mercari-price-suggestion-challenge/sample_submission.csv.7z -y
# !7z x ../input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip -y




The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 1522 kB of archives.
After this operation, 5755 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip amd64 16.02+dfsg-6 [358 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip-full amd64 16.02+dfsg-6 [1164 kB]
Fetched 1522 kB in 0s (5039 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package p7zip.
(Reading database ... 95327 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-6_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [  9%] [#####.....................................................] 8Unpacking p7zip (16.02+d

In [3]:
# train_path = './train.tsv'
# test_path = './test.tsv'
test2_path = './test_stg2.tsv'

In [4]:
import pandas as pd

# train_df = pd.read_csv(train_path, sep='\t')
# test_df = pd.read_csv(test_path, sep = '\t')

# train_df = train_df.head(8000)
# test_df = test_df.head(6000)

test_stg2_df = pd.read_csv(test2_path, sep = '\t')
test_stg2_df.head(5)

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


### Price Processing

In [5]:
def remove_invalid_price(df):
    if 'price' in df:
#         df = df[df.price >= 5]  # Removing the prodicts with prices less than 5
#         df = df[df.price <= 2000]  # Removing the prodicts with prices more than 2000
#     return df
        df.drop(df[df['price'] < 5].index, inplace = True)
        df.drop(df[df['price'] > 2000].index, inplace = True)

In [6]:
# remove_invalid_price(train_df)
# train_df[(train_df.price < 5) & (train_df.price > 2000)]

In [7]:
def create_log_price(df):
    if 'price' in df:
        df['log_prices']= np.log(df['price'].values)

In [8]:
# create_log_price(train_df)
# train_df.head(5)

### Handling null values


In [9]:
def fill_missing_values(df):
    """
    Description:
    Filling null values in all columns.

    Input: Dataframe with null values
    Output: Dataframe with no null values
    """
    df['name'].fillna('unk_name', inplace=True)
    df['category_name'].fillna('unk_cat', inplace=True)
    df['brand_name'].fillna('unk_brand', inplace=True)
    df['item_description'].fillna('unk_descr', inplace=True)
    return df

### Splitting categories

In [10]:
# tách caterory ra làm 3 category con
def split_categories(category):
    try:
        sub_cat_1,sub_cat_2,sub_cat_3 = category.split("/")
        return sub_cat_1,sub_cat_2,sub_cat_3
    except:
        return ("None","None","None")

# sử dụng hàm split_categories để tách category ra làm 3 category con và tạo thành 3 cột mới : 'main_category', 'sub_cat_1','sub_cat_2'
def create_split_categories(data):
    if 'category_name' in data:
        data['general_cat'],data['subcat_1'],data['subcat_2']= zip(*data['category_name'].\
                                                                    apply(lambda x: split_categories(x)))
#         data.drop('category_name',axis='columns', inplace=True)

In [11]:
# fill_missing_values(train_df)
# create_split_categories(train_df)

### Text Processing

In [12]:
import re
import string

def decontract_text(phrase):
    """
    This utility funciton will be used as a part of preprocessing the text.
    It will expand the contracted words. For eg: won't -> will not, I'm -> I am.
    """
    phrase = str(phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def preprocess_descriptive_text_column(text_data):
    """
    Description:
    This function will process the text data.

    Input: original strings [in list form]
    Output: processed strings [in list form]
    """
    # https://gist.github.com/sebleier/554280
    # we are removing the negative words from the stop words list: 'no', 'nor', 'not', 'shouldn't, won't, etc.
    stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
                "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
                'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
                'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
                'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
                'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
                'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
                'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
                'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
                'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
                's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
                've', 'y']

    processed_text = []
    for sentance in tqdm(text_data):
        sent = decontract_text(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        processed_text.append(sent.lower().strip())
    return np.array(processed_text)

In [13]:
def preprocess_cols(df):
    df['name'] = preprocess_descriptive_text_column(df['name'].values)
    df['brand_name'] = df['brand_name'].apply(lambda x: str(x).lower())
    df['item_description'] = preprocess_descriptive_text_column(df['item_description'].values)
    df['general_cat'] = preprocess_descriptive_text_column(df['general_cat'].values)
    df['subcat_1'] = preprocess_descriptive_text_column(df['subcat_1'].values)
    df['subcat_2'] = preprocess_descriptive_text_column(df['subcat_2'].values)

In [14]:
# train_df

### Fill Missing Brand Name

In [15]:
# inspired from https://www.kaggle.com/gspmoreira/cnn-glove-single-model-private-lb-0-41117-35th
def concat_categories(x):
        return set(x.values)

def brand_guesser(df):
    """
    Description:
    This function is used to guess the missing brand name.
    It will check for an existing brand name mentioned in the item name section. 
    We want out guess to be as close to actual as possible, \
    hence we will also check the category of the brand name that is guessed with the already mentioned category of that product. 
    If the category matches, then only we will fill that guessed brand name.

    Inputs: dataframe with missing brand names
    Output: dataframe with filled brand names
    """
    existing_brands = df[df['brand_name'] != 'unk_brand']['brand_name'].unique()
    brand_names_categories = dict(df[df['brand_name'] != 'unk_brand'][['brand_name','category_name']].astype('str').groupby('brand_name').agg(concat_categories).reset_index().values.tolist())
    # In the above line, we are creating dictionary of brand name->category, wherever the brand name is missing.
    # This will be helpful to us during guessing the missing brand names.
    filled_brands = []
    for row in tqdm(df[['brand_name','name','category_name']].values):
        found=False
        if row[0]=='unk_brand':
            for brand in existing_brands:
                if brand in row[1] and row[2] in brand_names_categories[brand] :
                    filled_brands.append(brand)
                    found=True
                    break
            if not found:
                filled_brands.append('unk_brand')
        else:
            filled_brands.append(row[0])

    df['brand_name']=filled_brands
    return df

In [16]:
def fill_missing_brands(df):
    print('Unknown brand before guessing:', df['brand_name'].value_counts()['unk_brand'])
    df = brand_guesser(df)
    print('Unknown brand after guessing:', df['brand_name'].value_counts()['unk_brand'])

### Adding Sentiment score

In [17]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


def get_sentiments_feature(df):
    """
    Description:
    This function will calculate the sentiment scores of each item description.
    It will calculate four scores for each sentence; compund, negetive, neutral, positive

    Input: Dataframe
    Output: sentiment scores for all the product's description
    """
    sentiments = []
    neg = []
    neu = []
    pos = []
    sid = SentimentIntensityAnalyzer()
    for desc in tqdm(df['item_description'].apply(str).values):
        ss = sid.polarity_scores(desc)
        sentiments.append([ss['neg'],ss['neu'],ss['pos']])
        neg.append(ss['neg'])
        neu.append(ss['neu'])
        pos.append(ss['pos'])
#     return np.array(sentiments)
    return neg, neu, pos

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
# sentiments_feature = get_sentiments_feature(train_df)

In [19]:
# neg, neu, pos = sentiments_feature
# neg

In [20]:
# train_df['desc_neg'] = neg
# train_df['desc_neu'] = neu
# train_df['desc_pos'] = pos
# train_df

In [21]:
def add_sentiment_scores(df):
    neg, neu, pos = get_sentiments_feature(df)
    df['desc_neg'] = neg
    df['desc_neu'] = neu
    df['desc_pos'] = pos

In [22]:
# train_df.isnull().sum()

### Expensive brand

In [23]:
# ##print 90 to 100 percentile values with step size of 1. 
# unique_gen_cat = train_df['general_cat'].unique()
# cat_threshold_percetile = {}
# for gc_ in unique_gen_cat: 
#     print(gc_,"\n-------------")
#     for i in range(90,101):
#         print(i,"th percentile is",np.percentile(train_df[train_df['general_cat']==gc_].groupby('brand_name').median().sort_values("price", ascending=False)['price'].reset_index()['price'].values, i))
#     cat_threshold_percetile[gc_]=np.percentile(train_df[train_df['general_cat']==gc_].groupby('brand_name').median().sort_values("price", ascending=False)['price'].reset_index()['price'].values, 95)
# #     print()

In [24]:
# expensive_brands = []
# unique_gen_cat = train_df['general_cat'].unique()
# for gc_ in unique_gen_cat:
#     temp_df = train_df[train_df['general_cat']==gc_][['brand_name','price']].values
#     for row in temp_df:
#         if row[1]>=cat_threshold_percetile[gc_]:
#             expensive_brands.append(row[0])
# expensive_brands = list(set(expensive_brands))
# try:
#     expensive_brands.remove('unk_brand')
# except:
#     pass

In [25]:
# expensive_brands

In [26]:
def add_expensive_brands(df):
    ##print 90 to 100 percentile values with step size of 1. 
    unique_gen_cat = df['general_cat'].unique()
    cat_threshold_percetile = {}
    for gc_ in unique_gen_cat: 
#         print(gc_,"\n-------------")
#         for i in range(90,101):
#             print(i,"th percentile is",np.percentile(df[df['general_cat']==gc_].groupby('brand_name').median().sort_values("price", ascending=False)['price'].reset_index()['price'].values, i))
        cat_threshold_percetile[gc_]=np.percentile(train_df[train_df['general_cat']==gc_].groupby('brand_name').median().sort_values("price", ascending=False)['price'].reset_index()['price'].values, 95)
    #     print()
    
    expensive_brands = []
    
    unique_gen_cat = train_df['general_cat'].unique()
    for gc_ in unique_gen_cat:
        temp_df = train_df[train_df['general_cat']==gc_][['brand_name','price']].values
        for row in temp_df:
            if row[1]>=cat_threshold_percetile[gc_]:
                expensive_brands.append(row[0])
    expensive_brands = list(set(expensive_brands))
    try:
        expensive_brands.remove('unk_brand')
    except:
        pass
    
    df['is_expensive'] = df['brand_name'].apply(lambda x: 1 if x in expensive_brands else 0)

In [27]:
# train_df

### Adding len of desrciption and name

In [28]:
from sklearn.preprocessing import StandardScaler
def get_len_feature(col_series, scaler_text_len=None):
    """
    Description:
    This funciton will calculate the word count of the item name and standardize it.

    Input: Series, fitted scaler[during inference]
    Output: standardized name length for each product and object of the fitted scaler
    """
    text_len = col_series.apply(lambda x: len(x.split()))
    if scaler_text_len==None:
        scaler_text_len = StandardScaler()
        scaler_text_len.fit(text_len.values.reshape(-1, 1))
    text_len = scaler_text_len.transform(text_len.values.reshape(-1, 1))
    return text_len, scaler_text_len

In [29]:
# name_len, scaler_name_len =  get_len_feature(train_df['name'])
# desc_len, scaler_desc_len =  get_len_feature(train_df['item_description'])

In [30]:
def add_len_features(df):
    name_len, scaler_name_len =  get_len_feature(df['name'])
    desc_len, scaler_desc_len =  get_len_feature(df['item_description'])
    df['scaler_name_len'] = name_len
    df['scaler_desc_len'] = desc_len

In [31]:
def combine_processes(df):
    print('Removing invalid price ...')
    remove_invalid_price(df)
    print('Completed \n')
    
    print('Creating log price ...')
    create_log_price(df)
    print('Completed \n')
    
    print('Filling NaN values ...')
    fill_missing_values(df)
    print('Completed \n')
    
    print('Splitting categories ...')
    create_split_categories(df)
    print('Completed \n')
    
    print('Filling unknown brands ...')
    fill_missing_brands(df)
    print('Completed \n')
    
    print('Cleaning text columns ...')
    preprocess_cols(df)
    print('Completed \n')
    
    print('Adding sentiment scores ...')
    add_sentiment_scores(df)
    print('Completed \n')
    
    print('Adding expensive brands ...')
    add_expensive_brands(df)
    print('Completed \n')
    
    print('Adding description len and name len ...')
    add_len_features(df)
    print('Completed \n')

In [32]:
# combine_processes(train_df)

In [33]:
# train_df

In [34]:
# train_df[(train_df['price'] < 5) | (train_df['price'] > 2000)]

In [35]:
# import seaborn as sns
# corr = train_df.corr()
# heatmap = sns.heatmap(corr)
# heatmap

In [36]:
# corr

In [37]:
# train_df.isnull().sum()

In [38]:
# compression_train = dict(method='zip', archive_name='train_processed.csv')  
# train_df.to_csv('train_out.zip', index=False, compression=compression_train)
# train_df.to_csv('train_processed.csv', index=False)

In [39]:
# # Save to pkl file to avoid missing data
# with open('./train_processed.pkl', 'wb') as f:
#     pickle.dump(train_df, f)

In [40]:
# combine_processes(test_df)
# test_df

In [41]:
# # compression_test = dict(method='zip', archive_name='test_processed.csv')  
# # test_df.to_csv('test_out.zip', index=False, compression=compression_test) 
# with open('./test_processed.pkl', 'wb') as f:
#     pickle.dump(test_df, f)

In [42]:
# train_df.isnull().sum()
test_stg2_df.isnull().sum()

test_id                    0
name                       0
item_condition_id          0
category_name          14833
brand_name           1476490
shipping                   0
item_description           6
dtype: int64

In [43]:
# compression_test = dict(method='zip', archive_name='test_processed.csv')  
# test_df.to_csv('test_out.zip', index=False, compression=compression_test) 
with open('./test_stg2_processed.pkl', 'wb') as f:
    pickle.dump(test_stg2_df, f)